In [1]:
import sys
import json
sys.path.append('..')
from common import *
from analysis.model_performances import *
from questions_construction.questions import *
from copy import deepcopy
from helpers import *
import pandas as pd


TO_PRETTY |= {(WITHOUT_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Baseline', 
              (WITHOUT_RANDOM_SUB, WITH_RAMIFICATIONS): 'Baseline + R.',
             (WITH_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Obfus. Baseline',
             (WITH_RANDOM_SUB, WITH_RAMIFICATIONS): 'Obfus. Baseline + R.',
}

In [85]:
questions_by_id = {d[OUT_OBJ_ID]: d for d in open_jsonl(f'{DATA_PATH}/test_data.paraphrased.cleaned.jsonl')}
override = True

answer_type = TRUE_FALSE_ANSWER_TYPE
answer_response_type = f'{answer_type}.{ACCURACY_SCORE_KEY}' #f'{TRUE_FALSE_ANSWER_TYPE}.{ACCURACY_SCORE_KEY}' #
stats_save_dir = f'{STATISTICS_PATH}.trial_run.ED'


substitution = WITHOUT_RANDOM_SUB
ramification = WITHOUT_RAMIFICATIONS
prompt_type = ZERO_SHOT_PROMPT_KEY #FEW_SHOT_3_PROMPT_KEY #
models = ['gpt-4o', 'llama_8b','llama_70b', 'llama_8b.finetuned_tf']

data_all = []
for model_name in models:
    model_results_dir = f'{PROJECT_PATH}/data/prompting_results/{ramification}/{prompt_type}/{model_name}.jsonl'
    model_results = open_jsonl(model_results_dir)
    data_all_inst = data_all_single_run(questions_by_id, model_results, substitution, ramification, model_name,prompt_type)
    data_all.extend(data_all_inst)

In [91]:
data_all[0]

{'question_id': '784689cc-3f7c-4a22-824a-ffc0e4c90483',
 'lebel': 'package p1 is in vehicle t1',
 'prompt': '[DOMAIN DESCRIPTION]\nLoading a package onto a truck or an airplane is only possible if both the package and the vehicle are at the same location. Once loaded, the package is no longer at the location but inside the vehicle. Unloading from a truck or airplane is only possible when the vehicle is at a location and the package is inside. Unloading transfers the package from the vehicle to the location. Driving a truck is only possible between locations within the same city, moving it from the source to the destination. Flying an airplane is possible when it is in a location, and flying moves it from the source to the destination.\n\n[INITIAL CONDITIONS]\nInitially, Airport l0_0 is situated in city c0, while airport l0_1 is also located in city c0. Airport l1_0 is found in city c1. At airport l0_0, you can find object a0, and object p3 is also located there. Airport l0_1 is home to

In [102]:
def to_df_by_models(data_all, prompt_type,models, ramifications, subs,
                      score_type=ACCURACY_SCORE_KEY, 
                      question_category = ALL_QUESTION_CATEGORIES_KEY,
                      domain = ALL_DOMAINS_KEY,
                      answer_type=TRUE_FALSE_ANSWER_TYPE,
                      plan_length=19):

    # index = []
    data_for_df = []    
    for fluent_type in FLUENT_TYPES_LIST:
        # index.append()
        data_columns = {}
        data_columns['fluent type'] = TO_PRETTY[fluent_type]
        for model in models:
            data = filter_multi_selector_modified(data_all, ramifications, model, prompt_type, answer_type, subs, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
            stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model, prompt_type, domain, subs, score_type=score_type)
            res_obj = stats.compute()
            if res_obj:
                mean = res_obj['result']
                sem = None
                if res_obj['result_other']:
                    sem = res_obj['result_other']['sem']
                final_res = (mean, sem)
            else:
                final_res = (None, None)
            final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
            final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
            # print(final_res)
            data_columns[TO_PRETTY.get(model, model)] = final_res
        data_for_df.append(data_columns)
    return pd.DataFrame(data_for_df)

def to_df_by_models_sign(data_all, prompt_type,models, ramifications, subs,
                      score_type=ACCURACY_SCORE_KEY, 
                      question_category = ALL_QUESTION_CATEGORIES_KEY,
                      domain = ALL_DOMAINS_KEY,
                      answer_type=TRUE_FALSE_ANSWER_TYPE,
                      plan_length=19):

    # index = []
    data_for_df = []    
    for fluent_sign in POS_NEG_FLUENTS_KEY_LIST:
        # index.append()
        data_columns = {}
        data_columns['fluent sign'] = fluent_sign
        for model in models:
            data = filter_multi_selector_modified(data_all, ramifications, model, prompt_type, answer_type, subs, plan_length, [(OUT_OBJ_FLUENT_SIGN_QUESTION, {fluent_sign})])
            stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model, prompt_type, domain, subs, score_type=score_type)
            res_obj = stats.compute()
            if res_obj:
                mean = res_obj['result']
                sem = None
                if res_obj['result_other']:
                    sem = res_obj['result_other']['sem']
                final_res = (mean, sem)
            else:
                final_res = (None, None)
            final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
            final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
            # print(final_res)
            data_columns[TO_PRETTY.get(model, model)] = final_res
        data_for_df.append(data_columns)
    return pd.DataFrame(data_for_df)

In [103]:
# def to_df_by_sub_ram_neg_pos(data_all, model_name, prompt_type,
#                       score_type=ACCURACY_SCORE_KEY, 
#                       question_category = ALL_QUESTION_CATEGORIES_KEY,
#                       domain = ALL_DOMAINS_KEY,
#                       answer_type=TRUE_FALSE_ANSWER_TYPE,
#                       plan_length=19):
# 
#     # index = []
#     data_for_df = []    
#     for is_pos_fluent in [True, False]:
#         # index.append()
#         data_columns = {}
#         data_columns['fluent type'] = is_pos_fluent
#         for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
#             for ramifications in [WITHOUT_RAMIFICATIONS, WITH_RAMIFICATIONS]:
#                 data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, subs, plan_length, [(OUT_OBJ_IS_POS_FLUENT_QUESTION, {is_pos_fluent})])
#                 stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, subs, score_type=score_type)
#                 res_obj = stats.compute()
#                 if res_obj:
#                     mean = res_obj['result']
#                     sem = None
#                     if res_obj['result_other']:
#                         sem = res_obj['result_other']['sem']
#                     not_corrupted = res_obj['stats']['num_not_corrupted']
#                     final_res = (mean, sem, not_corrupted)
#                 else:
#                     final_res = (None, None, None)
#                 final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
#                 final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
#                 # print(final_res)
#                 data_columns[TO_PRETTY[(subs, ramifications)]] = final_res
#         data_for_df.append(data_columns)
#     return pd.DataFrame(data_for_df)
# 
# def to_df_by_sub_ram_all(data_all, model_name, prompt_type,
#                       score_type=ACCURACY_SCORE_KEY, 
#                       question_category = ALL_QUESTION_CATEGORIES_KEY,
#                       domain = ALL_DOMAINS_KEY,
#                       answer_type=TRUE_FALSE_ANSWER_TYPE,
#                       plan_length=19):
# 
#     # index = []
#     data_for_df = []    
#     for fluent_type in FLUENT_TYPES_LIST:
#         # index.append()
#         data_columns = {}
#         data_columns['fluent type'] = TO_PRETTY[fluent_type]
#         for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
#             for ramifications in [WITHOUT_RAMIFICATIONS]: #WITH_RAMIFICATIONS
#                 data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, subs, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
#                 stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, subs, score_type=score_type)
#                 res_obj = stats.compute()
#                 if res_obj:
#                     mean = res_obj['result']
#                     sem = None
#                     if res_obj['result_other']:
#                         sem = res_obj['result_other']['sem']
#                     not_corrupted = res_obj['stats']['num_not_corrupted']
#                     final_res = (mean, sem, not_corrupted)
#                 else:
#                     final_res = (None, None, None)
#                 final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
#                 final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
#                 # print(final_res)
#                 data_columns[TO_PRETTY[(subs, ramifications)]] = final_res
#         data_for_df.append(data_columns)
#     return pd.DataFrame(data_for_df)


# Subs and Ramfications

In [104]:
plan_length = 19
df = to_df_by_models_sign(data_all, prompt_type, models, ramification, substitution, answer_type = answer_type, plan_length=plan_length)
df
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

,fluent sign,gpt-4o,llama_8b,llama_70b,llama_8b.finetuned_tf
0,pos,${77.48}_{3.4}$,${72.86}_{5.32}$,${76.99}_{3.96}$,${85.6}_{3.14}$
1,neg,${73.85}_{3.85}$,${58.21}_{6.03}$,${71.31}_{4.09}$,${77.19}_{3.93}$
2,pos_neg,${73.84}_{1.79}$,${57.1}_{2.81}$,${69.61}_{1.93}$,${88.5}_{1.27}$


In [105]:
latex_table_all = to_latex_table(df, 'some caption', index=False)
print(latex_table_all)


\begin{table}[h!]
\begin{adjustbox}{width=\textwidth,center}
\begin{tabular}{lllll}
\toprule
fluent sign &           gpt-4o &         llama_8b &        llama_70b & llama_8b.finetuned_tf \\
\midrule
        pos &  ${77.48}_{3.4}$ & ${72.86}_{5.32}$ & ${76.99}_{3.96}$ &       ${85.6}_{3.14}$ \\
        neg & ${73.85}_{3.85}$ & ${58.21}_{6.03}$ & ${71.31}_{4.09}$ &      ${77.19}_{3.93}$ \\
    pos_neg & ${73.84}_{1.79}$ &  ${57.1}_{2.81}$ & ${69.61}_{1.93}$ &       ${88.5}_{1.27}$ \\
\bottomrule
\end{tabular}

\end{adjustbox}
\caption{some caption}
\label{table:}
\end{table}



/Users/paveldolin/dev/research/completed/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=index, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [80]:
df_pos_neg = to_df_by_sub_ram_neg_pos(data_all, model_name, prompt_type, plan_length=plan_length)
df_pos_neg
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

NameError: name 'OUT_OBJ_IS_POS_FLUENT_QUESTION' is not defined

# By Models

In [45]:
score_type=ACCURACY_SCORE_KEY, 
question_category = ALL_QUESTION_CATEGORIES_KEY,
domain = ALL_DOMAINS_KEY,
answer_type=TRUE_FALSE_ANSWER_TYPE,
plan_length=1

data_for_df = []    
for fluent_type in FLUENT_TYPES_LIST:
    # index.append()
    data_columns = {}
    data_columns['fluent type'] = TO_PRETTY[fluent_type]
    for model in models:
        data = filter_multi_selector_modified(data_all, ramification, model, prompt_type, answer_type, substitution, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
        print(len(data))
        stats = TrueFalseStatsCustom(data, plan_length, question_category, ramification, model, prompt_type, domain, substitution, score_type=score_type)
        res_obj = stats.compute()
        if res_obj:
            mean = res_obj['result']
            sem = None
            if res_obj['result_other']:
                sem = res_obj['result_other']['sem']
            final_res = (mean, sem)
        else:
            final_res = (None, None)
        final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
        final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
        # print(final_res)
        data_columns[TO_PRETTY.get(model, model)] = final_res
    data_for_df.append(data_columns)

0
0
0
0
0
0
0
0
0
0
0
0


In [64]:
data = filter_multi_selector_modified(data_all, ramification, 'llama_8b', prompt_type, 'free_answer', substitution, 1, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])

In [65]:
data

[]

In [29]:
prompt_type

'zero_shot'

In [54]:
prompt_type

'zero_shot'

In [55]:
substitution

'without_random_sub'

In [56]:
data_all[0]

{'question_id': '784689cc-3f7c-4a22-824a-ffc0e4c90483',
 'domain_name': 'logistics',
 'instance_id': 'Instance_2',
 'question_category': 'composite',
 'question_name': 'iter_6_question_2',
 'fluent_type': 'all_fluents',
 'answer_type': 'free_answer',
 'question': 'Given the initial condition, the following actions are planned to be performed: at airport l1_1, package p2 is loaded in truck t1, at airport l1_1, package p1 is loaded in truck t1, truck t1 is driven to airport l1_0 from airport l1_1 in city c1, in city c1, truck t0 is driven from airports l1_1 to l0_1, from truck t1 package p1 is unloaded at airport l1_0, in city c0, truck t0 is driven from airports l0_1 to l0_0, at airport l0_0, package p3 is loaded in airplane a0, airplane a0 flies from airports l0_0 to l1_0, from airplane a0 package p3 is unloaded at airport l1_0 and package p3 is loaded in truck t1 at airport l1_0 to reach the current state. What are the valid properties of the state that do not involve negations for p1